In [1]:
%pip install pyarrow


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import urllib.request as urllib
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os

In [3]:
# open pba csv as df

pba_df = pd.read_csv('Data/PBA--Commissioners-Cup_2024_averages.csv')

In [4]:
pba_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   #         132 non-null    int64  
 1   Player    132 non-null    object 
 2   Team      132 non-null    object 
 3   GP        132 non-null    int64  
 4   Position  132 non-null    object 
 5   MPG       132 non-null    float64
 6   PPG       132 non-null    float64
 7   FGM       132 non-null    float64
 8   FGA       132 non-null    float64
 9   FG%       132 non-null    float64
 10  3PM       132 non-null    float64
 11  3PA       132 non-null    float64
 12  3P%       132 non-null    float64
 13  FTM       132 non-null    float64
 14  FTA       132 non-null    float64
 15  FT%       132 non-null    float64
 16  ORB       132 non-null    float64
 17  DRB       132 non-null    float64
 18  RPG       132 non-null    float64
 19  APG       132 non-null    float64
 20  SPG       132 non-null    float6

In [5]:
# open stats_fix.csv as df

df = pd.read_csv('Data/stats_fix.csv')

In [6]:
# show info with all columns

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 46 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Number        129 non-null    int64  
 1   Name          129 non-null    object 
 2   Team          129 non-null    object 
 3   Position      114 non-null    object 
 4   PPG           129 non-null    float64
 5   RPG           129 non-null    float64
 6   APG           129 non-null    float64
 7   SPG           129 non-null    float64
 8   BPG           129 non-null    float64
 9   TOPG          129 non-null    float64
 10  FPG           129 non-null    float64
 11  FDPG          129 non-null    float64
 12  +/-           129 non-null    float64
 13  EFFPG         129 non-null    float64
 14  FG%           129 non-null    object 
 15  3PT%          129 non-null    object 
 16  FT%           129 non-null    object 
 17  MPG_x         129 non-null    float64
 18  NAME          129 non-null    

In [7]:
# rename columns

df.rename(columns={
    'Name':'Player',
    'TOPG':'TOV',
    'FPG':'PF',
    'MPG_x':'MPG',
    ' 3PT FGs %': '3P%',
    ' 2PT FGs %': 'FG%'
    }, inplace=True)

# divide DEF and OFF by GP and add as DRB and ORB

df['DRB'] = df['DEF REBOUNDS'] / df['GP']
df['ORB'] = df['OFF REBOUNDS'] / df['GP']

# drop FT% and rename FTs % to FT%

df.drop(columns=['FT%'], inplace=True)
df.rename(columns={'FTs %':'FT%'}, inplace=True)

# split 3PT FGs M/A to 3PM and 3PA, 2PT FGs M/A to 2PM and 2PA, and FTs M/A to FTM and FTA

df['3PM'] = df['3PT FGs M/A'].str.split('/').str[0].astype(float) / df['GP']
df['3PA'] = df['3PT FGs M/A'].str.split('/').str[1].astype(float) / df['GP']
df['2PM'] = df['2PT FGs M/A'].str.split('/').str[0].astype(float) / df['GP']
df['2PA'] = df['2PT FGs M/A'].str.split('/').str[1].astype(float) / df['GP']
df['FTM'] = df['FTs M/A'].str.split('/').str[0].astype(float) / df['GP']
df['FTA'] = df['FTs M/A'].str.split('/').str[1].astype(float) / df['GP']

# create FGM and FGA by adding 3PM and 2PM, 3PA and 2PA

df['FGM'] = df['3PM'] + df['2PM']
df['FGA'] = df['3PA'] + df['2PA']

# drop all columns that don't match pba_df

# get pba_df columns

pba_columns = pba_df.columns

# get df columns

df_columns = df.columns

# get columns that don't match

columns_to_drop = [column for column in df_columns if column not in pba_columns]

columns_to_drop


['Number',
 'FDPG',
 '+/-',
 'EFFPG',
 '3PT%',
 'NAME',
 'MIN',
 'MPG_y',
 'STARTS',
 '3PT FGs M/A',
 '2PT FGs M/A',
 '2PT FGs %',
 'FGs %',
 'DEFENDED M/A',
 'DEFENDED %',
 'OPEN M/A',
 'OPEN %',
 'FTs M/A',
 'HI',
 'LO',
 'DEF REBOUNDS',
 'OFF REBOUNDS',
 'AST',
 'STL',
 'BLK',
 'OFF TOs',
 'DEF TOs',
 'TECH',
 'UF',
 'TOTAL',
 '2PM',
 '2PA']

In [8]:
# drop the columns

df.drop(columns=columns_to_drop, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Player    129 non-null    object 
 1   Team      129 non-null    object 
 2   Position  114 non-null    object 
 3   PPG       129 non-null    float64
 4   RPG       129 non-null    float64
 5   APG       129 non-null    float64
 6   SPG       129 non-null    float64
 7   BPG       129 non-null    float64
 8   TOV       129 non-null    float64
 9   PF        129 non-null    float64
 10  FG%       129 non-null    object 
 11  MPG       129 non-null    float64
 12  GP        129 non-null    int64  
 13  3P%       129 non-null    float64
 14  FT%       129 non-null    float64
 15  DRB       129 non-null    float64
 16  ORB       129 non-null    float64
 17  3PM       129 non-null    float64
 18  3PA       129 non-null    float64
 19  FTM       129 non-null    float64
 20  FTA       129 non-null    float6

In [9]:
# add an ID column named '#'

df['#'] = df.index +1 

# convert FG% to float

df['FG%'] = df['FG%'].str.replace('%', '').astype(float)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Player    129 non-null    object 
 1   Team      129 non-null    object 
 2   Position  114 non-null    object 
 3   PPG       129 non-null    float64
 4   RPG       129 non-null    float64
 5   APG       129 non-null    float64
 6   SPG       129 non-null    float64
 7   BPG       129 non-null    float64
 8   TOV       129 non-null    float64
 9   PF        129 non-null    float64
 10  FG%       129 non-null    float64
 11  MPG       129 non-null    float64
 12  GP        129 non-null    int64  
 13  3P%       129 non-null    float64
 14  FT%       129 non-null    float64
 15  DRB       129 non-null    float64
 16  ORB       129 non-null    float64
 17  3PM       129 non-null    float64
 18  3PA       129 non-null    float64
 19  FTM       129 non-null    float64
 20  FTA       129 non-null    float6

In [11]:
# sort columns to match pba_df

df = df[pba_columns]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   #         129 non-null    int64  
 1   Player    129 non-null    object 
 2   Team      129 non-null    object 
 3   GP        129 non-null    int64  
 4   Position  114 non-null    object 
 5   MPG       129 non-null    float64
 6   PPG       129 non-null    float64
 7   FGM       129 non-null    float64
 8   FGA       129 non-null    float64
 9   FG%       129 non-null    float64
 10  3PM       129 non-null    float64
 11  3PA       129 non-null    float64
 12  3P%       129 non-null    float64
 13  FTM       129 non-null    float64
 14  FTA       129 non-null    float64
 15  FT%       129 non-null    float64
 16  ORB       129 non-null    float64
 17  DRB       129 non-null    float64
 18  RPG       129 non-null    float64
 19  APG       129 non-null    float64
 20  SPG       129 non-null    float6

In [12]:
# find duplicates on key=player

# show all columns in print df

pd.set_option('display.max_columns', None)

duplicates = df[df.duplicated(subset='Player', keep=False)]

duplicates

,#,Player,Team,GP,Position,MPG,PPG,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,DRB,RPG,APG,SPG,BPG,TOV,PF
12,13,"Faye, Adama",UST,13,C,30.57,11.4,4.615385,10.153846,45.1,0.000000,0.538462,0.00,2.153846,5.615385,38.36,2.000000,10.230769,12.2,1.0,0.4,2.2,3.8,3.1
13,14,"Faye, Adama",UST,13,C,30.57,11.4,4.615385,10.153846,45.1,0.000000,0.538462,0.00,2.153846,5.615385,38.36,2.000000,10.230769,12.2,1.0,0.4,2.2,3.8,3.1
14,15,"Pangilinan, Miguel",UST,14,G,25.46,6.6,2.142857,7.428571,28.8,1.214286,4.357143,27.87,1.071429,1.571429,68.18,1.857143,2.785714,4.6,1.4,0.9,0.1,1.6,2.2
15,16,"Pangilinan, Miguel",UST,14,G,25.46,6.6,2.142857,7.428571,28.8,1.214286,4.357143,27.87,1.071429,1.571429,68.18,1.857143,2.785714,4.6,1.4,0.9,0.1,1.6,2.2
16,17,"Lazarte, Ivan",UST,11,G,15.18,5.5,1.727273,6.090909,28.4,0.545455,2.818182,19.35,1.545455,2.545455,60.71,1.181818,1.636364,2.8,0.8,0.8,0.0,0.5,1.3
17,18,"Lazarte, Ivan",UST,11,G,15.18,5.5,1.727273,6.090909,28.4,0.545455,2.818182,19.35,1.545455,2.545455,60.71,1.181818,1.636364,2.8,0.8,0.8,0.0,0.5,1.3
18,19,"Manalang, Paul",UST,13,G,25.08,4.5,1.307692,6.769231,19.3,0.846154,4.769231,17.74,1.076923,1.384615,77.78,0.384615,2.307692,2.7,2.8,0.6,0.0,2.0,2.5
19,20,"Manalang, Paul",UST,13,G,25.08,4.5,1.307692,6.769231,19.3,0.846154,4.769231,17.74,1.076923,1.384615,77.78,0.384615,2.307692,2.7,2.8,0.6,0.0,2.0,2.5
20,21,"Manaytay, Christian",UST,14,F,19.08,3.9,1.571429,4.214286,37.3,0.000000,0.571429,0.00,0.785714,1.714286,45.83,2.000000,2.928571,4.9,1.4,0.9,0.3,1.9,2.6
21,22,"Manaytay, Christian",UST,14,F,19.08,3.9,1.571429,4.214286,37.3,0.000000,0.571429,0.00,0.785714,1.714286,45.83,2.000000,2.928571,4.9,1.4,0.9,0.3,1.9,2.6


In [13]:
# keep one copy of the duplicates

df.drop_duplicates(subset='Player', keep='first', inplace=True)

duplicates = df[df.duplicated(subset='Player', keep=False)]

duplicates

,#,Player,Team,GP,Position,MPG,PPG,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,DRB,RPG,APG,SPG,BPG,TOV,PF


In [14]:
# drop the # column

df.drop(columns='#', inplace=True)

# readd the # column

df['#'] = df.index +1

In [15]:
# save df as UAAP_2024_averages.csv

df.to_csv('Data/UAAP_2024_averages.csv', index=False)


In [16]:
# read csv to check

df = pd.read_csv('Data/UAAP_2024_averages.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Player    115 non-null    object 
 1   Team      115 non-null    object 
 2   GP        115 non-null    int64  
 3   Position  101 non-null    object 
 4   MPG       115 non-null    float64
 5   PPG       115 non-null    float64
 6   FGM       115 non-null    float64
 7   FGA       115 non-null    float64
 8   FG%       115 non-null    float64
 9   3PM       115 non-null    float64
 10  3PA       115 non-null    float64
 11  3P%       115 non-null    float64
 12  FTM       115 non-null    float64
 13  FTA       115 non-null    float64
 14  FT%       115 non-null    float64
 15  ORB       115 non-null    float64
 16  DRB       115 non-null    float64
 17  RPG       115 non-null    float64
 18  APG       115 non-null    float64
 19  SPG       115 non-null    float64
 20  BPG       115 non-null    float6